# Получение данных 

**Изучим полученные данные**

Импортируем библиотеку Pandas и получим сведения из таблицы данных (первые 10 строк)

In [118]:
import pandas as pd

data = pd.read_csv('music_project.csv', index_col=0)
data.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday



**Получим общую информацию данных таблицы**

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 4.0+ MB



**Изучим эту информацию**

Всего в таблице 7 столбцов с типами данных - **object**

* userID — идентификатор пользователя;
* Track — название трека;
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Также в некоторых столбца есть различия в количестве значений. Следовательно они содержат **пропущенные значения** (чаще всего это **None** или **NaN**).

**Выводы**

В полученных данных мы имеем две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. 
Также по полученным данным из столбца **genre** мы сможем определить самые популярные жанры прослушиваемой музыки.


# Предобработка данных

Для начала мы должны проверить пропущенные значения **NaN**, а после избавиться от них. Также проверить таблицу на наличие дубликатов.

Перед тем как приступать к решению этих проблем, мы должны отредактировать название столбцов таблицы, чтобы привести их к правильному виду, руководствуясь следующими правилами:

* отсутствие пробелов;
* состоят из одного слова (использовать символы нижнего подчеркивания);
* написаны на одном языке и в одном регистре (нижнем);
* сообщают, какого рода информация содержится в каждом столбце.

In [8]:
data.columns

Index(['Unnamed: 0', '  userID', 'Track', 'artist', 'genre', '  City  ',
       'time', 'Day'],
      dtype='object')

Используем метод **set_axis()** для изменения названий столбцов, где
аргументы - список новых названий столбцов,
axis со значением columns для изменений в столбцах,
inplace со значением True для изменения структуры данных

In [35]:
data.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], axis='columns', inplace=True)
data.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

Используем методы **isnull()** или **isna()** для определения пропущенных значений; в сочетании с методом **sum()** — подсчёт пропущенных значений.

In [36]:
data.isna().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64


Столбцы с названием трека, исполнителем и жанром песни содержат пропуски. Такие пустые значения сообщают нам, что по некоторым произведениям доступна не вся информация. Причины такой проблемы могут быть различные, поэтому необходимо разобрать каждый отдельный случай и понять в чём причина пропусков.

Для начала мы можем заменить пропуски в этих столбцах на любое значение, например **'unknown'**, после чего проверим, содержит ли таблица пропуски. 

Используем метод **fillna()** для заполнения пропущенных значений. Аргументом является значение, которое будет присвоено вместо пропуска.

In [40]:
data['track_name'] = data['track_name'].fillna('unknown')
data['artist_name'] = data['artist_name'].fillna('unknown')

data.isna().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Разберёмся со столбцом жанра песни. Удалим все пропущенные значения в этом столбце. Для этого используем метод **dropna()**.


аргумент **subset** — названия столбцов, где нужно искать пропуски

In [41]:
data.dropna(subset=['genre_name'], inplace=True)

data.isna().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

Разбираемся с наличием дубликатов в данных.

In [44]:
data.duplicated().sum()

3756

Метод **drop_duplicates()** для удаления дубликатов. Вместе с повторяющимися строками удаляет их индексы. Поэтому вызывается с методом **reset_index()**

In [46]:
data = data.drop_duplicates().reset_index(drop=True)

data.duplicated().sum()

0



Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Необходимо определить список неявных дубликатов в столбце **genre_name**, когда один и тот же жанр может быть записан разными словами.

Используем метод **unique()** для просмотра всех уникальных значений в столбце

In [48]:
genres_list = data['genre_name'].unique()

Создаём функцию для поиска таких неявных дубликатов

In [49]:
def f_genres(name):
    counter = 0
    for i in genres_list:
        if i == name:
            counter += 1
    return counter

In [51]:
f_genres('rock')

1

Создадим функцию для исправления неправильного написания жанра в таблице

In [59]:
def func_genre(data, genre_bad, genre_good):
    data['genre_name'] = data['genre_name'].replace(genre_bad, genre_good)
    n = data[data['genre_name']==genre_bad]['genre_name'].count()
    return n

Проверим наличие неправильного написания жанра *hiphop* и проверим нашу функцию

In [61]:
f_genres('hip')

1

In [65]:
func_genre(data, 'hip', 'hiphop')

0

Получаем общую информацию о данных. Убеждаемся, что чистка выполнена успешно.

In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60125 entries, 0 to 60124
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60125 non-null  object
 1   track_name   60125 non-null  object
 2   artist_name  60125 non-null  object
 3   genre_name   60125 non-null  object
 4   city         60125 non-null  object
 5   time         60125 non-null  object
 6   weekday      60125 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


## Вывод

После предобработки данных выяснилось, что мы избавились от пропусков в данных таблицы, а также избавились от дубликатов. Наши действия позволят вести дальнейший анализ точнее. Имена столбцов теперь корректны и удобны для дальнейшей работы.


# Анализ данных



Пришло время проверить одну из гипоте, что в разных городах пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для проверки необходимо сгруппировать данные по городу и подсчитать композиции. Для этого применяем метод **count()**. Для группировки используем метод **groupby()**

In [69]:
data.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18233
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Питере. Это может говорить нам, что в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.


In [70]:
data.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17777
Name: genre_name, dtype: int64

можно сделать вывод, что в среду пользователи меньше слушают музыку, например, по причине большей нагруженности на работе.

Создадим функцию для определения количества прослушиваний в конкретный день недели в выбранном городе.

In [85]:
def f_city_day(data, city, day):
    g_list = data[(data['city'] == city) & (data['weekday'] == day)]
    n = g_list['genre_name'].count()
    return n

In [86]:
# для Москвы
result11 = f_city_day(data,'Moscow','Monday')
result12 = f_city_day(data,'Moscow','Wednesday')
result13 = f_city_day(data,'Moscow','Friday')
# для Питера
result21 = f_city_day(data,'Saint-Petersburg','Monday')
result22 = f_city_day(data,'Saint-Petersburg','Wednesday')
result23 = f_city_day(data,'Saint-Petersburg','Friday')

Сведём полученную информацию в одну таблицу.

In [87]:
tcolumns = ['city', 'monday', 'wednesday', 'friday']
trows = ['Moscow',result11,result12,result13],['Saint-Petersburg',result21,result22,result23]
table = pd.DataFrame(data=trows, columns=tcolumns)
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6912,5802


## Вывод

Действительно в этих городах музыку слушают по-разному. В Москве больше всего прослушиваний в пятницу и чуть меньше в понедельник, а в среду же идёт большой спад по числу прослушиваний, когда в Питере наоборот, в среду достигается пик прослушиваний, а в понедельник и пятницу число прослушиваний почти одинаково меньше.




## Какие жанры преобладают в разных городах


Определим какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получим таблицы данных по Москве moscow_general и по Санкт-Петербургу spb_general.

In [88]:
moscow_general = data[data['city'] =='Moscow']
spb_general = data[data['city'] =='Saint-Petersburg']

Создаём функцию **genre_weekday()**, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то Метод **sort_values(by = 'название столбца')** для сортировки таблицы по указанному столбцу.

In [101]:
def genre_weekday(data, day, time1, time2):
    g_list = data[ (data['weekday'] == day) & (data['time'] > time1) & (data['time'] < time2) ]
    g_list_sorted = g_list.groupby(['genre_name']).count().sort_values(by='artist_name',ascending=False)['artist_name'].head(10)
    return g_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [106]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')


genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: artist_name, dtype: int64

In [107]:
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: artist_name, dtype: int64

In [108]:
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: artist_name, dtype: int64

In [109]:
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: artist_name, dtype: int64

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен **pop**. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит **jazz** и **rusrap**, а в Москве жанр **world**.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр **world**.

## Вывод

Жанр **pop** безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

## Проверка последней гипотезы

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

Сгруппируем таблицу moscow_general по жанру, сосчитаем численность композиций каждого жанра методом **count()**, отсортируем в порядке убывания и сохраним результат в таблице moscow_genres.

Просмотрим первые 10 строк этой новой таблицы.

In [113]:
moscow_genres = moscow_general.groupby(['genre_name']).count()['artist_name'].sort_values(ascending=False)
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: artist_name, dtype: int64

Сгруппируем таблицу spb_general по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат в таблице spb_genres.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [114]:
spb_genres = spb_general.groupby(['genre_name']).count()['artist_name'].sort_values(ascending=False)
spb_genres.head(10)

genre_name
pop            2430
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: artist_name, dtype: int64

## Вывод

В Москве абсолютно популярным жанром является **pop**, но также и в Санкт-Петербурге этот жанр является лидирующим в списках. 
А рэп, вопреки предположению, занимает в обоих городах низкие позиции.

# Результаты исследования

### Рабочие гипотезы:

1. музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

2. списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

3. население двух городов предпочитает разные музыкальные жанры.

### Общие результаты

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза подтверждена и третья не подтверждена.

In [115]:
data.head(10)

,user_id,track_name,artist_name,genre_name,city,time,weekday
0,FFB692EC,Kamigata To Boots,unknown,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,unknown,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,unknown,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,unknown,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,unknown,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,unknown,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,unknown,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,unknown,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


В колонке **'artist_name'** есть незаполненные значения *unknown*.

In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60125 entries, 0 to 60124
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60125 non-null  object
 1   track_name   60125 non-null  object
 2   artist_name  60125 non-null  object
 3   genre_name   60125 non-null  object
 4   city         60125 non-null  object
 5   time         60125 non-null  object
 6   weekday      60125 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


In [117]:
data.shape

(60125, 7)